In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#import helpers

In [76]:
def load_data(path_dataset,sub_sample=True, add_outlier=False):
    """Load data and convert it to the metric system."""
    data = np.genfromtxt(
        path_dataset, delimiter=",", dtype=None, encoding=None)
    labels = data[0,:]
    data = data[1:,:]
    data[:,1][data[:,1]=='s']=0
    data[:,1][data[:,1]=='b']=1
    data = np.asarray(data, dtype=float)
    data[:,:][data[:,:]==-999]=None
    return data, np.array(labels)

Prediction of :
- type s is assigned value 0
- type b is assigned value 1

In [77]:
# Load data
train_data, labels = load_data('train.csv')

In [86]:
# remove the 'Prediction' column (string thus useless)
print(train_data)

[[ 1.00000e+05  0.00000e+00  1.38470e+02 ...  1.24000e+00 -2.47500e+00
   1.13497e+02]
 [ 1.00001e+05  1.00000e+00  1.60937e+02 ...          nan          nan
   4.62260e+01]
 [ 1.00002e+05  1.00000e+00          nan ...          nan          nan
   4.42510e+01]
 ...
 [ 3.49997e+05  0.00000e+00  1.05457e+02 ...          nan          nan
   4.19920e+01]
 [ 3.49998e+05  1.00000e+00  9.49510e+01 ...          nan          nan
   0.00000e+00]
 [ 3.49999e+05  1.00000e+00          nan ...          nan          nan
   0.00000e+00]]


## Logistic regression

- data (np.array): Dataset of shape (N, D) 
- labels (np.array): Labels of shape (N, ) 
- w (np.array): Weights of logistic regression model of shape (D, ) 

In [ ]:
def sigmoid(t):
    """ Sigmoid function
    
    Args:
        t (np.array): Input data of shape (N, )
        s
    Returns:
        np.array: Probabilites of shape (N, ), where each value is in [0, 1].
    """
    return 1/(1+np.exp(-t))